In [ ]:
!pip install pandas transformers guidance numpy evaluate rouge-score nltk py-rouge accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import json
import nltk
import numpy as np
from guidance import gen
from evaluate import load
import csv
import nltk.translate.bleu_score as bleu
from rouge import Rouge
from transformers import BertTokenizer, BertModel
import torch
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import warnings
from typing import Any, Dict, Tuple
import re

warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Dataset

- id: unique identifier of the example
- source: original model input for paraphrase generation
- hyp1: first alternative paraphrase of the source
- hyp2: second alternative paraphrase of the source
- label: hyp1 or hyp2, based on which of those has been annotated as hallucination
- type: hallucination category assigned.
  - Possible values: addition, named-entity, number, conversion, date, tense, negation, gender, pronoun, antonym, natural

## Trial dataset

This is a small list of examples, provided to help the participants get familiar with the task. Each example contains the following fields:
- id
- source
- hyp1
- hyp2
- label
- type

In [ ]:
from datasets import load_dataset

#load the trial data for both English and Swedish
trial_ds = load_dataset("Eloquent/HalluciGen-PG", name="trial")

#load the trial data only for Swedish
trial_ds_sv = load_dataset("Eloquent/HalluciGen-PG", name="trial", split="trial_swedish")

print(trial_ds.keys())

trial_df_en = trial_ds['trial_english'].to_pandas()
trial_df_se= trial_ds['trial_swedish'].to_pandas()

print('English:')

display(trial_df_en.head())

print('Swedish')
display(trial_df_se.head())

dict_keys(['trial_swedish', 'trial_english'])
English:


,id,source,type,hyp1,hyp2,label
0,0,The population has declined in some 210 of the...,antonym,In the majority of Sweden's 280 municipalities...,In the majority of Sweden's 280 municipalities...,hyp1
1,1,The draft agenda as drawn up by the Conference...,negation,The Conference of Presidents hasn't distribute...,The Conference of Presidents has distributed t...,hyp1
2,2,I am always grateful for comments and suggesti...,antonym,I think I need to listen to everyone's views i...,I think I need to listen to everyone's views i...,hyp1
3,3,The fact is that a key omission from the propo...,named entity,Agenda 2030 does not include a chapter on rene...,Agenda 2000 does not include a chapter on rene...,hyp1
4,4,"We need quite specific legislative proposals, ...",natural,Legislative proposals based on the Treaty of A...,Legislative proposals based solely on the Trea...,hyp2


Swedish


,id,source,type,hyp1,hyp2,label
0,0,"Men intäkterna från mjukvarulicenser, ett mått...",number,"Intäkter från programvarulicenser, en metrik s...","Intäkter från programvarulicenser, en metrik s...",hyp2
1,1,Hongkong-universitet samarbetar med universite...,natural,University of Hong Kong samarbetar med olika k...,University of Hong Kong samarbetar med olika k...,hyp1
2,2,Län med befolkningsminskning kommer att vara V...,named entity,"Vermillion, Posey och Madison är län som komme...","Vermillion, Posey och Marion är län som kommer...",hyp2
3,3,Google presenterar en prototyp för en självkör...,addition,En prototyp för en självkörande bil presentera...,Enligt ett blogginlägg från Google har företag...,hyp2
4,4,Lagförslaget säger att en kvinna som genomgår ...,negation,Förslaget stadgar att en kvinna som genomgår e...,Förslaget stadgar att en kvinna som genomgår e...,hyp1


In [ ]:
print("Source:", trial_df_en.loc[0, 'source'])
print("Hyp1:", trial_df_en.loc[0, 'hyp1'])
print("Hyp2:", trial_df_en.loc[0, 'hyp2'])


Source: The population has declined in some 210 of the 280 municipalities in Sweden, mainly in inland central and northern Sweden.
Hyp1: In the majority of Sweden's 280 municipalities, the population has gone up.
Hyp2: In the majority of Sweden's 280 municipalities, the population has gone down.


In [ ]:
#load the test data for the generation step in both English and Swedish
data = load_dataset("Eloquent/HalluciGen-PG", "test_generation")


print(data.keys())

data_df_en = data['test_generation_english'].to_pandas()
data_df_se= data['test_generation_swedish'].to_pandas()


dict_keys(['test_generation_swedish', 'test_generation_english'])


In [ ]:
data_df_en

,id,source
0,0,It has enabled us to support and encourage an ...
1,1,"Therefore, I am calling for an increase in the..."
2,2,"In other words, a person may be prevented from..."
3,3,None of my 34 amendments were adopted and my a...
4,4,The final key feature of the report is its ori...
...,...,...
114,114,I am referring to the current vote but in rela...
115,115,When I became a Member of the European Parliam...
116,116,"Mr President, let me express my thanks for the..."
117,117,We can replace 16 out of 626 Members of the Eu...


In [ ]:
data_df_se

,id,source
0,0,Kvinnor kommer att möta högre bilförsäkringspr...
1,1,"Rörelseintäkterna var 1,45 miljarder dollar, e..."
2,2,"Mandela tillbaka på sjukhus i ""allvarligt men ..."
3,3,Egypten fryser Muslimska brödraskapets tillgån...
4,4,"P-pillret, som de kallar ""polypiller"", skulle ..."
...,...,...
71,71,Du kommer att komma på saker! Jag kommer att b...
72,72,Påven uppmanar till åtgärder mot klimatförändr...
73,73,Egyptens ledare inför undantagstillstånd i tre...
74,74,En svartvit bild av en katt som ligger på en m...


In [ ]:
# Replace "'" with '//"'
data_df_en['source'] = data_df_en['source'].str.replace("'", "\\'")

In [ ]:
selected_value = data_df_en.loc[12, 'source']
print(selected_value)

B4-0188/99 by Mrs d\'Ancona, on behalf of the PSE Group, on the death penalty against Greg Summers - Texas, USA; Leonard Peltier


In [ ]:
data_df_se['source'] = data_df_se['source'].str.replace("'", "\\'")

In [ ]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.7 MB/s eta 0:00:00


In [ ]:
!pip install -U "transformers==4.38.1" --upgrade
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 102.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0


In [ ]:
#load model Gemma

from transformers import AutoTokenizer, pipeline
import torch

model = "google/gemma-7b-it"
model_id = "google/gemma-7b-it"
dtype = torch.bfloat16

# use quantized model
pipeline = pipeline(
    "text-generation",
    model=model,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True}
    },
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

# Prompt Version 2

In [ ]:
def generate_paraphrase(source):
    # Generate the prompt

    answer_format = {"hyp+": "", "hyp-": ""}

    user_prompt = f'''
      <start_of_turn>user
      As an AI model, your task is to generate two paraphrases based on the given source text. The first paraphrase, labeled as 'hyp+', should be supported by the source text. The second paraphrase, labeled as 'hyp-', should not be supported by the source text.

      Here's an example to illustrate this:
      Source: The population has declined in some 210 of the 280 municipalities in Sweden, mainly in inland central and northern Sweden.
      hyp+: In the majority of Sweden's 280 municipalities, the population has gone down.
      This is a paraphrase that is supported by the source. It's saying essentially the same thing as the source: the population has decreased in most municipalities. The wording is different, but the meaning is the same. Hence, it's labeled as 'hyp+'.
      hyp-: In the majority of Sweden's 280 municipalities, the population has gone up.
      This is a paraphrase that is not supported by the source. It's saying the opposite of what the source says: the population has increased in most municipalities. This contradicts the information in the source. Hence, it's labeled as 'hyp-'.

      Now, given the source text below, generate 'hyp+' and 'hyp-' paraphrases and provide the result in the following format: {answer_format}

      Source: {source}
      <end_of_turn>

      Result:
      <start_of_turn>model
      '''

    messages = [
      #{"role": "user", "content": "You are a text generator. You are specialized in paraphrasing texts."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=150,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    generated_paraphrase = outputs[0]["generated_text"][len(prompt):]

    return generated_paraphrase

In [ ]:
def generate_paraphrase_se(source):
    # Generate the prompt

    answer_format = {"hyp+": "", "hyp-": ""}

    user_prompt = f'''
      <start_of_turn>användare
      Som en AI-modell är din uppgift att generera två parafraser baserade på den angivna källtexten. Den första parafrasen, märkt som 'hyp+', ska stödjas av källtexten. Den andra parafrasen, märkt som 'hyp-', ska inte stödjas av källtexten.

      Här är ett exempel för att illustrera detta:
      Källa: Intäkterna från mjukvarulicenser, ett mått som finansanalytiker följer noga, minskade med 21 procent till 107,6 miljoner dollar.
      hyp+: Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 21 procent till ett belopp av 107,6 miljoner dollar.
      Detta är en parafras som stöds av källan. Den säger i princip samma sak som källan: intäkterna har minskat i de flesta kommunerna. Formuleringen är annorlunda, men betydelsen är densamma. Därför märks den som 'hyp+'.
      hyp-: Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 42 procent till ett belopp av 107,6 miljoner dollar.
      Detta är en parafras som inte stöds av källan. Den säger motsatsen till vad källan säger: intäkterna har ökat i de flesta kommunerna. Detta motsäger informationen i källan. Därför märks den som 'hyp-'.

      Nu, med den angivna källtexten nedan, generera 'hyp+' och 'hyp-' parafraser och ge resultatet i följande format: {answer_format}

      Källa: {source}
      <end_of_turn>

      Resultat:
      <start_of_turn>modell
      '''


    messages = [
      #{"role": "user", "content": "You are a text generator. You are specialized in paraphrasing texts."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=150,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    generated_paraphrase = outputs[0]["generated_text"][len(prompt):]

    return generated_paraphrase


In [ ]:
import json
import re

def extract_json_data(result_text):
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]

    # Replace single quotes used as delimiters with double quotes

    result_text = result_text.replace("{'", '{"').replace("': '", '": "').replace("', '", '", "').replace("'}", '"}')

    # Convert the JSON data to a Python dictionary
    data_dict = json.loads(result_text)

    print('result_text:',result_text)

    return data_dict

In [ ]:
source = "My mom's food is better than my sister's."
result = generate_paraphrase(source)
extract_json_data(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'hyp+': 'My mom's food is better than my sister's food.', 'hyp-': 'My sister's food is better than my mom's food.'}

The source text states that "My mom's food is better than my sister's". Therefore, the paraphrase hypothesis hyp+ is supported by the source text. The paraphrase hypothesis hyp- is not supported by the source text.
result_text: {"hyp+": "My mom's food is better than my sister's food.", "hyp-": "My sister's food is better than my mom's food."}


{'hyp+': "My mom's food is better than my sister's food.",
 'hyp-': "My sister's food is better than my mom's food."}

## Generate dataset

In [ ]:

import os
import numpy as np
import pandas as pd

def save_results(generate_paraphrase_func, prompt, df, csv_filename):
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']

            try:
                # Generate paraphrase
                result = generate_paraphrase_func(source)
                extracted_hyp = extract_json_data(result)

                hyp_support = extracted_hyp['hyp+']
                print(hyp_support)
                hyp_hallu = extracted_hyp['hyp-']
                print(hyp_hallu)
                print('source:',source)

                # Update DataFrame with new values
                chunk.at[index, 'hyp+'] = hyp_support
                chunk.at[index, 'hyp-'] = hyp_hallu
                chunk.at[index, 'prompt'] = prompt
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        display(df)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gemma/results_gemma_en_v2.csv"

prompt = '''
       <start_of_turn>user
      As an AI model, your task is to generate two paraphrases based on the given source text. The first paraphrase, labeled as 'hyp+', should be supported by the source text. The second paraphrase, labeled as 'hyp-', should not be supported by the source text.
      Here's an example to illustrate this:
      Source: The population has declined in some 210 of the 280 municipalities in Sweden, mainly in inland central and northern Sweden.
      hyp+: In the majority of Sweden's 280 municipalities, the population has gone down.
      This is a paraphrase that is supported by the source. It's saying essentially the same thing as the source: the population has decreased in most municipalities. The wording is different, but the meaning is the same. Hence, it's labeled as 'hyp+'.
      hyp-: In the majority of Sweden's 280 municipalities, the population has gone up.
      This is a paraphrase that is not supported by the source. It's saying the opposite of what the source says: the population has increased in most municipalities. This contradicts the information in the source. Hence, it's labeled as 'hyp-'.
      Now, given the source text below, generate 'hyp+' and 'hyp-' paraphrases and provide the result in the following format: {answer_format}
      Source: {source}
      <end_of_turn>
      Result:
      <start_of_turn>model
      '''

save_results(generate_paraphrase, prompt, data_df_en, csv_filename)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Gen text:  {'hyp+': 'It has enabled us to foster a collaborative environment and provide opportunities for individuals to enhance their skills across Europe.', 'hyp-': 'It has enabled us to provide financial assistance and create a platform for sharing experiences across Europe.'}
result_text: {"hyp+": "It has enabled us to foster a collaborative environment and provide opportunities for individuals to enhance their skills across Europe.", "hyp-": "It has enabled us to provide financial assistance and create a platform for sharing experiences across Europe."}
It has enabled us to foster a collaborative environment and provide opportunities for individuals to enhance their skills across Europe.
It has enabled us to provide financial assistance and create a platform for sharing experiences across Europe.
source: It has enabled us to support and encourage an exchange of experiences and to pursue activities to raise the level of competence throughout Europe.
Gen text:  {'hyp+': 'An increas

,id,source,hyp+,hyp-,prompt
0,0,It has enabled us to support and encourage an ...,It has enabled us to foster a collaborative en...,It has enabled us to provide financial assista...,\n <start_of_turn>user\n As an AI m...
1,1,"Therefore, I am calling for an increase in the...",An increase in premiums for leaf tobacco is ne...,An decrease in premiums for leaf tobacco is ne...,\n <start_of_turn>user\n As an AI m...
2,2,"In other words, a person may be prevented from...",A person may be prevented from coming near the...,A person may be able to get closer to their vi...,\n <start_of_turn>user\n As an AI m...
3,3,None of my 34 amendments were adopted and my a...,"None of my proposed amendments were accepted, ...","All of my proposed amendments were adopted, an...",\n <start_of_turn>user\n As an AI m...
4,4,The final key feature of the report is its ori...,The report's final key feature is its orientat...,The report's final key feature is its orientat...,\n <start_of_turn>user\n As an AI m...
5,5,It is high time an appropriate level of social...,A significant level of social protection shoul...,The current employment patterns in Europe are ...,\n <start_of_turn>user\n As an AI m...
6,6,"These sums, however, have still not produced t...",The substantial fall in the average earnings o...,The substantial fall in the average earnings o...,\n <start_of_turn>user\n As an AI m...
7,7,"Mr President, the approach adopted by the rapp...",The approach adopted by the rapporteur to the ...,The approach adopted by the rapporteur to the ...,\n <start_of_turn>user\n As an AI m...
8,8,"The Berlin summit, which will focus solely on ...","The Berlin summit, focused on Agenda 2000, hol...","The Berlin summit, focused on Agenda 2000, hol...",\n <start_of_turn>user\n As an AI m...
9,9,"As a result, I cannot promise the chairman of ...",I cannot guarantee the chairman of the Committ...,There will be 100 % funding for the financial ...,\n <start_of_turn>user\n As an AI m...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'hyp+': 'Although Amendments Nos 22 to 25 fall short of adequate measures, we still voted for them.', 'hyp-': 'Despite their inadequacy, we cast our votes for Amendments Nos 22 to 25.'}
result_text: {"hyp+": "Although Amendments Nos 22 to 25 fall short of adequate measures, we still voted for them.", "hyp-": "Despite their inadequacy, we cast our votes for Amendments Nos 22 to 25."}
Although Amendments Nos 22 to 25 fall short of adequate measures, we still voted for them.
Despite their inadequacy, we cast our votes for Amendments Nos 22 to 25.
source: We voted for Amendments Nos 22 to 25, even though they do not take adequate measures to prevent these problems from happening.


...

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'hyp+': 'The pay and working conditions of navigation professionals should be comparable to other occupations.', 'hyp-': 'The pay and working conditions of navigation professionals should be significantly higher than other occupations.'}
result_text: {"hyp+": "The pay and working conditions of navigation professionals should be comparable to other occupations.", "hyp-": "The pay and working conditions of navigation professionals should be significantly higher than other occupations."}
The pay and working conditions of navigation professionals should be comparable to other occupations.
The pay and working conditions of navigation professionals should be significantly higher than other occupations.
source: An important factor here is that the pay and other working conditions of those employed in navigation should be in line with those of other occupations.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'hyp+': 'Without the chairmanship skills of the Deputy Prime Minister, John Prescott, the Conference would have produced more hot air than it prevented.',
'hyp-': 'Without the chairmanship skills of the Deputy Prime Minister, John Prescott, the Conference would have produced less hot air than it prevented.'}
Skipping row 111 due to error in data extraction.
Gen text:  {'hyp+': 'The Commission is no longer able to handle the ever-increasing tasks without additional resources.', 'hyp-': 'The Commission is prepared to accept ever-increasing tasks without receiving the means to execute them.'}
result_text: {"hyp+": "The Commission is no longer able to handle the ever-increasing tasks without additional resources.", "hyp-": "The Commission is prepared to accept ever-increasing tasks without receiving the means to execute them."}
The Commission is no longer able to handle the ever-increasing tasks without additional resources.
The Commission is prepared to accept ever-increasing 

,id,source,hyp+,hyp-,prompt
0,0,It has enabled us to support and encourage an ...,It has enabled us to foster a collaborative en...,It has enabled us to provide financial assista...,\n <start_of_turn>user\n As an AI m...
1,1,"Therefore, I am calling for an increase in the...",An increase in premiums for leaf tobacco is ne...,An decrease in premiums for leaf tobacco is ne...,\n <start_of_turn>user\n As an AI m...
2,2,"In other words, a person may be prevented from...",A person may be prevented from coming near the...,A person may be able to get closer to their vi...,\n <start_of_turn>user\n As an AI m...
3,3,None of my 34 amendments were adopted and my a...,"None of my proposed amendments were accepted, ...","All of my proposed amendments were adopted, an...",\n <start_of_turn>user\n As an AI m...
4,4,The final key feature of the report is its ori...,The report's final key feature is its orientat...,The report's final key feature is its orientat...,\n <start_of_turn>user\n As an AI m...
...,...,...,...,...,...
114,114,I am referring to the current vote but in rela...,"The current vote, as well as the upcoming vote...",The current vote is being discussed in relatio...,\n <start_of_turn>user\n As an AI m...
115,115,When I became a Member of the European Parliam...,When I became a Member of the European Parliam...,When I became a Member of the European Parliam...,\n <start_of_turn>user\n As an AI m...
116,116,"Mr President, let me express my thanks for the...","Mr President, I appreciate your remarks and ex...","Mr President, I regret that you have not heard...",\n <start_of_turn>user\n As an AI m...
117,117,We can replace 16 out of 626 Members of the Eu...,We can replace a significant portion of the Me...,We can never replace the Members of the Europe...,\n <start_of_turn>user\n As an AI m...


In [ ]:
# save a csv file in the task format

import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gemma/results_gemma_en_v2.csv')

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'hyp+', 'hyp-']]

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/final_gemma_en_v2.csv', index=False)

# Swedish

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gemma/results_gemma_se_v2.csv"

prompt = '''
       <start_of_turn>användare
    Med tanke på källan nedan, generera en parafras-hypotes hyp+ som stöds av källan och en andra parafras hyp- som inte stöds av källan.
    Ge resultatet i följande format: {answer_format}
    Källa: {source}
    <end_of_turn>
    Resultat:
    <start_of_turn>modell
      '''

save_results(generate_paraphrase_se, prompt, data_df_se, csv_filename)

In [ ]:
# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_gemma/results_gemma_se_v2.csv')

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'hyp+', 'hyp-']]

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/final_gemma_se_v2.csv', index=False)